# Домашнее задание к лекции "Продвинутый Pandas"

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

*   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
*   для источников paid и email из России ставим ad;
*   для источников paid и email не из России ставим other;
*   все остальные варианты берём из traffic_source без изменений.

In [ ]:
# Вариант 1

import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')
log.head()

def get_source_type(row):
    if row.traffic_source in ('yandex', 'google'):
        return 'organic'
    elif row.traffic_source in ('paid', 'email'):
        return 'ad' if row.region == 'Russia' else 'other'
    else:
        return row.traffic_source

log['source_type'] = log.apply(get_source_type, axis=1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [ ]:
# Вариант 2 - более быстрый

import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')
log.head()

log.loc[log.traffic_source.isin(('yandex', 'google')), 'source_type'] = 'organic'
log.loc[(log.traffic_source.isin(('paid', 'email'))) & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[(log.traffic_source.isin(('paid', 'email'))) & (log.region != 'Russia'), 'source_type'] = 'other'
log.loc[~log.traffic_source.isin(('yandex', 'google', 'paid', 'email')), 'source_type'] = log['traffic_source']
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       18938 non-null  int64 
 1   visit_id        18938 non-null  object
 2   url             18938 non-null  object
 3   region          18938 non-null  object
 4   user_id         18938 non-null  object
 5   traffic_source  18938 non-null  object
 6   source_type     18938 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


## Задание 2

В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

1.   Прочитайте содержимое файла с датафрейм.
2.   Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
url = pd.read_csv('URLs.txt')
url.head()

news = url[url.url.str.contains('/\d{8}-')]
news.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
# Вариант 1 с сохранением промежуточных вычислений и объединением таблиц

ratings = pd.read_csv('ratings.csv')
ratings.head()

proper_users = ratings['userId'].value_counts().reset_index().query('count > 100') # Фильтруем пользователей, выставивших более 100 оценок
ltv = ratings.groupby('userId').apply(lambda row: row.timestamp.max() - row.timestamp.min(), include_groups=False).reset_index() # Считаем LTV для всех пользователей
ltv.rename(columns={0: 'ltv'}, inplace=True) # Переименовываем столбец с LTV
proper_ltv = proper_users.merge(ltv, how='left', on='userId') # Объединяем таблицы по подходящим пользователям
round(proper_ltv['ltv'].mean() / 86400) # Считаем среднее время жизни подходящих пользователей в днях

464

In [ ]:
# Вариант 2 - оптимизированный

ratings = pd.read_csv('ratings.csv')
ratings.head()

df = ratings.groupby('userId').agg({'timestamp': ['min', 'max', 'count']}).reset_index() # Группируем ДФ по необходимым параметрам
df['ltv'] = df['timestamp']['max'] - df['timestamp']['min'] # Считаем LTV для всех пользователей
round(df[df['timestamp']['count'] > 100]['ltv'].mean() / 86400) # Вычисляем средний LTV подходящих пользователей в днях

464

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам:

*   rzd - железнодорожные перевозки
*   auto - автомобильные перевозки
*   air - воздушные перевозки
*   client_base - адреса клиентов

Нужно сформировать две таблицы:

*   таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
*   аналогичную таблицу по типам выручки с указанием адреса клиента.

*Обратите внимание, что в процессе объединения таблиц данные не должны теряться.*

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

rzd
auto
air
client_base

total_revenue = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
total_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
total_info = total_revenue.merge(client_base, how='outer', on='client_id')
total_info

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
